# Model Experiments
Testing multiple architectures on a subset of training data

## Import Dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, models
from PIL import Image
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import random
from tqdm import tqdm

## Define Constants

In [ ]:
# Paths
TRAIN_DIR = Path('data/train')
EXPERIMENTS_DIR = Path('data/experiments')

# Training parameters
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
IMG_SIZE = 224
VAL_SPLIT = 0.2
EXPERIMENT_SAMPLE_SIZE = 0.2  # 20% of training data

# Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')

# Class names
CLASS_NAMES = ['healthy', 'multiple_diseases', 'rust', 'scab']
NUM_CLASSES = len(CLASS_NAMES)

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

## Create Experiments Dataset
Copy 20% of images from each class in train to experiments folder

In [ ]:
# Create experiments directory if it doesn't exist
if not EXPERIMENTS_DIR.exists():
    EXPERIMENTS_DIR.mkdir(parents=True, exist_ok=True)
    print(f'Created experiments directory: {EXPERIMENTS_DIR}')
    
    # Copy 20% of images from each class to experiments
    for class_name in CLASS_NAMES:
        train_class_dir = TRAIN_DIR / class_name
        exp_class_dir = EXPERIMENTS_DIR / class_name
        exp_class_dir.mkdir(parents=True, exist_ok=True)
        
        # Get all images in this class
        all_images = list(train_class_dir.glob('*.jpg'))
        
        # Calculate number of images to copy (20%)
        num_exp = int(len(all_images) * EXPERIMENT_SAMPLE_SIZE)
        
        # Randomly select images for experiments
        exp_images = random.sample(all_images, num_exp)
        
        # Copy images to experiments folder
        for img_path in exp_images:
            dest_path = exp_class_dir / img_path.name
            shutil.copy2(str(img_path), str(dest_path))
        
        print(f'{class_name}: Copied {num_exp} images to experiments ({len(all_images)} total in train)')
    
    print(f'\nExperiments dataset created successfully!')
else:
    print(f'Experiments directory already exists: {EXPERIMENTS_DIR}')
    # Show experiments set statistics
    for class_name in CLASS_NAMES:
        exp_class_dir = EXPERIMENTS_DIR / class_name
        if exp_class_dir.exists():
            num_exp_images = len(list(exp_class_dir.glob('*.jpg')))
            print(f'{class_name}: {num_exp_images} experiment images')

## Image Generators
Using experiments folder as training set with 20% validation split

In [ ]:
class PlantPathologyDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Data augmentation and normalization
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Prepare experiment data
exp_image_paths = []
exp_labels = []

for class_idx, class_name in enumerate(CLASS_NAMES):
    class_dir = EXPERIMENTS_DIR / class_name
    for img_path in class_dir.glob('*.jpg'):
        exp_image_paths.append(str(img_path))
        exp_labels.append(class_idx)

print(f'Experiment images: {len(exp_image_paths)}')

# Split into train and validation (20% validation)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    exp_image_paths, exp_labels, test_size=VAL_SPLIT, random_state=42, stratify=exp_labels
)

print(f'Train split: {len(train_paths)}')
print(f'Validation split: {len(val_paths)}')

# Create datasets
train_dataset = PlantPathologyDataset(train_paths, train_labels, train_transform)
val_dataset = PlantPathologyDataset(val_paths, val_labels, val_transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f'Created dataloaders: train={len(train_loader)} batches, val={len(val_loader)} batches')

## Training Functions

In [ ]:
def train_model(model, model_name, train_loader, val_loader, epochs, device):
    """
    Train a model and log results to tensorboard
    """
    # Setup tensorboard writer
    writer = SummaryWriter(f'runs/{model_name}')
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1}/{epochs}')
        print('-' * 50)
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in tqdm(train_loader, desc='Training'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc='Validation'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        # Log to tensorboard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }, f'best_{model_name}.pth')
            print(f'Best model saved with validation accuracy: {val_acc:.2f}%')
    
    writer.close()
    print(f'\n{model_name} training completed! Best validation accuracy: {best_val_acc:.2f}%')
    return best_val_acc

## Model 1: ResNet50

In [ ]:
print('\n' + '='*70)
print('Training ResNet50')
print('='*70)

# Load pretrained ResNet50
resnet50 = models.resnet50(pretrained=True)

# Freeze all layers
for param in resnet50.parameters():
    param.requires_grad = False

# Replace classifier
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, NUM_CLASSES)
)

resnet50 = resnet50.to(DEVICE)
print(f'ResNet50 loaded and moved to {DEVICE}')

# Train
resnet50_acc = train_model(resnet50, 'ResNet50', train_loader, val_loader, EPOCHS, DEVICE)

## Model 2: EfficientNet-B1

In [ ]:
print('\n' + '='*70)
print('Training EfficientNet-B1')
print('='*70)

# Load pretrained EfficientNet-B1
efficientnet_b1 = models.efficientnet_b1(pretrained=True)

# Freeze all layers
for param in efficientnet_b1.parameters():
    param.requires_grad = False

# Replace classifier
num_features = efficientnet_b1.classifier[1].in_features
efficientnet_b1.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, NUM_CLASSES)
)

efficientnet_b1 = efficientnet_b1.to(DEVICE)
print(f'EfficientNet-B1 loaded and moved to {DEVICE}')

# Train
efficientnet_b1_acc = train_model(efficientnet_b1, 'EfficientNet-B1', train_loader, val_loader, EPOCHS, DEVICE)

## Model 3: EfficientNet-B2

In [ ]:
print('\n' + '='*70)
print('Training EfficientNet-B2')
print('='*70)

# Load pretrained EfficientNet-B2
efficientnet_b2 = models.efficientnet_b2(pretrained=True)

# Freeze all layers
for param in efficientnet_b2.parameters():
    param.requires_grad = False

# Replace classifier
num_features = efficientnet_b2.classifier[1].in_features
efficientnet_b2.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, NUM_CLASSES)
)

efficientnet_b2 = efficientnet_b2.to(DEVICE)
print(f'EfficientNet-B2 loaded and moved to {DEVICE}')

# Train
efficientnet_b2_acc = train_model(efficientnet_b2, 'EfficientNet-B2', train_loader, val_loader, EPOCHS, DEVICE)

## Model 4: VGG16

In [ ]:
print('\n' + '='*70)
print('Training VGG16')
print('='*70)

# Load pretrained VGG16
vgg16 = models.vgg16(pretrained=True)

# Freeze all layers
for param in vgg16.parameters():
    param.requires_grad = False

# Replace classifier
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, NUM_CLASSES)
)

vgg16 = vgg16.to(DEVICE)
print(f'VGG16 loaded and moved to {DEVICE}')

# Train
vgg16_acc = train_model(vgg16, 'VGG16', train_loader, val_loader, EPOCHS, DEVICE)

## Model 5: DINOv2

In [ ]:
print('\n' + '='*70)
print('Training DINOv2')
print('='*70)

# Load pretrained DINOv2 (using torch.hub)
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

# Freeze all layers
for param in dinov2.parameters():
    param.requires_grad = False

# Add classifier
class DINOv2Classifier(nn.Module):
    def __init__(self, dinov2_model, num_classes):
        super().__init__()
        self.dinov2 = dinov2_model
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(384, num_classes)  # DINOv2 ViT-S/14 outputs 384 features
        )
    
    def forward(self, x):
        features = self.dinov2(x)
        return self.classifier(features)

dinov2_model = DINOv2Classifier(dinov2, NUM_CLASSES).to(DEVICE)
print(f'DINOv2 loaded and moved to {DEVICE}')

# Train
dinov2_acc = train_model(dinov2_model, 'DINOv2', train_loader, val_loader, EPOCHS, DEVICE)

## Results Summary

In [ ]:
print('\n' + '='*70)
print('FINAL RESULTS SUMMARY')
print('='*70)
print(f'ResNet50:         {resnet50_acc:.2f}%')
print(f'EfficientNet-B1:  {efficientnet_b1_acc:.2f}%')
print(f'EfficientNet-B2:  {efficientnet_b2_acc:.2f}%')
print(f'VGG16:            {vgg16_acc:.2f}%')
print(f'DINOv2:           {dinov2_acc:.2f}%')
print('='*70)

# Find best model
results = {
    'ResNet50': resnet50_acc,
    'EfficientNet-B1': efficientnet_b1_acc,
    'EfficientNet-B2': efficientnet_b2_acc,
    'VGG16': vgg16_acc,
    'DINOv2': dinov2_acc
}
best_model = max(results, key=results.get)
print(f'\nBest performing model: {best_model} ({results[best_model]:.2f}%)')
print('\nTo view tensorboard logs, run:')
print('tensorboard --logdir=runs')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs